# Código para seleção e criação do modelo

In [1]:
import numpy as np
import pandas as pd

In [2]:
def stratified_sample(df, col='CATEGORY', n_per_class=2):
    return df.groupby(col, group_keys=False).apply(
        lambda x: x.sample(min(len(x), n_per_class)))

# Carregamento dos dados

In [3]:
data = pd.read_csv('../1.scraping/products.tsv', sep='\t')
data.set_index('ID', inplace=True)
data['SMARTPHONE'] = (data.CATEGORY == 'celular-e-smartphone').astype(int)

stratified_sample(data)

,CATEGORY,SOURCE,TITLE,SMARTPHONE
ID,,,,
175,aparelho-de-telefone,https://www.buscape.com.br/com-fio-pro-eletron...,Com Fio Pro Eletronic Proks-5040,0
428,aparelho-de-telefone,https://www.buscape.com.br/redirect_prod?id=37...,Kit Telefone 2 Linhas Ts 5150 + 3 Ramais Ts 51...,0
725,capa-para-celular-e-smartphone,https://www.buscape.com.br/redirect_prod?id=46...,Capa para Galaxy Grand 2 Duos TV Raiden MP1311...,0
937,capa-para-celular-e-smartphone,https://www.buscape.com.br/redirect_prod?id=46...,Capa para Galaxy J5 Prime Calçada Copacabana T...,0
1352,celular-e-smartphone,https://www.buscape.com.br/smartphone-asus-zen...,Smartphone Asus ZenFone 5 Selfie ZC600KL 64GB ...,1
1291,celular-e-smartphone,https://www.buscape.com.br/smartphone-motorola...,Smartphone Motorola Moto X Force XT1580 32GB,1
1461,gabinete,https://www.buscape.com.br/redirect_prod?id=11...,Gabinete Flex Computer GAMER GM8001 BLACK S / ...,0
1652,gabinete,https://www.buscape.com.br/redirect_prod?id=11...,Gabinete ATX - AeroCool Cyclops ( c / janela )...,0
1967,jogos-de-rpg,https://www.buscape.com.br/redirect_prod?id=63...,Funko Pop Game Of Thrones: Wun Wun 6 ´,0


## Limpeza dos títulos

Devido a dificuldades em raspar de outras fontes, só pude coletar ofertas do buscapé.  
Um problema dessa fonte é que todos os produtos identificados como smartphones tinham a palavra smartphone ou celular no título.

In [4]:
print('Frequência das primeiras palavras de cada produto categorizado ' +
        f'como celular:')
phone_titles = data[data.CATEGORY == 'celular-e-smartphone'].TITLE
first_words = phone_titles.apply(lambda t: t.split()[0])

print(first_words.value_counts())

Frequência das primeiras palavras de cada produto categorizado como celular:
Smartphone    417
Celular        63
Name: TITLE, dtype: int64


Portanto realizei uma "limpeza" parcial dos títulos.  
Remover as palavras de todos os smartphones jogaria fora um atributo importante para a classificação,  
mas mantê-las seria quase incluir a variável resposta nas variáveis preditoras.

In [5]:
clean_titles = phone_titles.apply(lambda t: ' '.join(t.split()[1:]))
smartphones = data[data.CATEGORY == 'celular-e-smartphone']
trimmed_smartphones_idx = smartphones.sample(frac=0.4).index
data.loc[trimmed_smartphones_idx, 'TITLE'] = clean_titles

In [6]:
data[data.SMARTPHONE == 1].sample(10)

,CATEGORY,SOURCE,TITLE,SMARTPHONE
ID,,,,
1032,celular-e-smartphone,https://www.buscape.com.br/celular-bright-seni...,Bright Sênior 0485,1
1022,celular-e-smartphone,https://www.buscape.com.br/smartphone-samsung-...,Samsung Galaxy S7 SM-G930 32GB,1
1277,celular-e-smartphone,https://www.buscape.com.br/smartphone-motorola...,Motorola Moto Z 2 Play XT1710 64GB Power Edition,1
1134,celular-e-smartphone,https://www.buscape.com.br/smartphone-xiaomi-m...,Smartphone Xiaomi Mi A2 Lite 64GB,1
1234,celular-e-smartphone,https://www.buscape.com.br/smartphone-motorola...,Smartphone Motorola Moto G 3ª Geração XT1543 8GB,1
966,celular-e-smartphone,https://www.buscape.com.br/celular-blu-tank-ii...,BLU Tank II T193,1
1412,celular-e-smartphone,https://www.buscape.com.br/smartphone-sony-xpe...,Sony Xperia XZ,1
1336,celular-e-smartphone,https://www.buscape.com.br/celular-freecel-fre...,Celular Freecel Free Cross,1
1319,celular-e-smartphone,https://www.buscape.com.br/smartphone-lg-k10-k...,LG K10 K430TV 16GB,1


# Extração de features

Como na primeira parte do processo seletivo, tentei extrair features a partir de matches de expressões regulares.  
Todas as features definidas e expressões regulares correspondentes se encontram no arquivo `extract_features.py`.

In [7]:
from extract_features import get_df_attributes, attr_col_names

In [8]:
features = get_df_attributes(data)
stratified_sample(features)

,CATEGORY,SOURCE,TITLE,SMARTPHONE,re_smart,re_phone,re_celular,re_letra_num,re_capa,re_para,re_pelicula,re_plus,re_chip,re_MP
ID,,,,,,,,,,,,,,
131,aparelho-de-telefone,https://www.buscape.com.br/redirect_prod?id=37...,Kit Telefone Sem Fio Intelbras Ts40id Preto + ...,0,0,1,0,1,0,0,0,0,0,0
173,aparelho-de-telefone,https://www.buscape.com.br/com-fio-intelbras-c...,Com Fio Intelbrás CFA 4022,0,0,0,0,0,0,0,0,0,0,0
931,capa-para-celular-e-smartphone,https://www.buscape.com.br/redirect_prod?id=46...,Capa para Galaxy A5 2016 Minions 11 MP13082018,0,0,0,0,1,1,1,0,0,0,0
850,capa-para-celular-e-smartphone,https://www.buscape.com.br/redirect_prod?id=46...,Capa para Galaxy J1 2016 Planets and Friends M...,0,0,0,0,1,1,1,0,0,0,0
1104,celular-e-smartphone,https://www.buscape.com.br/smartphone-positivo...,Positivo Twist S S520,1,0,0,0,1,0,0,0,0,0,0
1003,celular-e-smartphone,https://www.buscape.com.br/smartphone-blackber...,Smartphone BlackBerry Curve 8520,1,1,1,0,0,0,0,0,0,0,0
1685,gabinete,https://www.buscape.com.br/redirect_prod?id=11...,Gabinete Bertolini Gourmet 4641 2 Portas 4 Gav...,0,0,0,0,0,0,0,0,0,0,0
1735,gabinete,https://www.buscape.com.br/redirect_prod?id=11...,Gabinete ATX - K - Mex CG - 01F6 ( c / janela ...,0,0,0,0,1,0,0,0,0,0,0
1969,jogos-de-rpg,https://www.buscape.com.br/redirect_prod?id=63...,Game Of Thrones - Boneco Pop Funko Tormund Gia...,0,0,0,0,0,0,0,0,0,0,0


In [9]:
clf_features = features[attr_col_names + ['SMARTPHONE']]
stratified_sample(clf_features, col='SMARTPHONE', n_per_class=5)

,re_smart,re_phone,re_celular,re_letra_num,re_capa,re_para,re_pelicula,re_plus,re_chip,re_MP,SMARTPHONE
ID,,,,,,,,,,,
3713,0,0,0,1,0,0,0,0,0,0,0
275,0,0,0,0,0,0,0,0,0,0,0
2962,0,0,0,1,0,0,0,0,0,0,0
2748,0,0,0,1,0,1,1,1,0,0,0
1732,0,0,0,1,0,0,0,0,0,0,0
1376,0,0,0,1,0,0,0,0,0,0,1
1325,0,0,0,1,0,0,0,0,0,0,1
980,0,0,0,1,0,0,0,0,0,0,1
1194,0,0,1,1,0,0,0,0,0,0,1


# Comparação de classificadores

Foram comparados os desempenhos do Perceptron e Naive Bayes sobre os atributos extraídos acima, e o do Bag of Words diretamente com os títulos (já "limpos").

In [10]:
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score

## Criação dos modelos

In [11]:
perceptron = Perceptron(max_iter=1e4)
naive_bayes = BernoulliNB(binarize=None)
bag_of_words = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('MNNB', MultinomialNB())
])

In [12]:
def eval_model(model, X, y):
    return cross_val_score(model, X, y, scoring='roc_auc', cv=10)


def print_results(name, cv_score):
    print(name)
    print(f'{np.mean(cv_score):.3f} +- {np.std(cv_score):.3f}')

## Avaliação dos resultados

In [13]:
perceptron_res = eval_model(perceptron, clf_features[attr_col_names].values, clf_features['SMARTPHONE'].values)

In [14]:
naive_bayes_res = eval_model(naive_bayes, clf_features[attr_col_names].values, clf_features['SMARTPHONE'].values)

In [15]:
bow_res = eval_model(bag_of_words, data.TITLE.values, data.SMARTPHONE.values)

In [16]:
print_results('Perceptron', perceptron_res)
print()
print_results('Naive Bayes', naive_bayes_res)
print()
print_results('Bag of Words', bow_res)

Perceptron
0.807 +- 0.202

Naive Bayes
0.841 +- 0.118

Bag of Words
0.995 +- 0.006


In [17]:
from scipy.stats import ttest_ind

test_result = ttest_ind(bow_res, naive_bayes_res)
print(f'p-value: {test_result.pvalue:.3f}')
if test_result.pvalue < 0.05:
    print(f'Classificadores com desempenhos distindos')
else:
    print(f'Não há evidências de que os classificadores tenham desempenhos diferentes')

p-value: 0.001
Classificadores com desempenhos distindos


## Escolha do modelo

Com base nos valores obtidos para a área sob a curva ROC e no teste de hipótese realizado acima, 
escolhi o modelo de Bag of Words para ser usado na classificação do dataset.

É válido notar que o formato e regularidade dos dados usados para treinamento não são representativos 
do conjunto dado como parte do teste, então mais testes rigorosos seriam necessários para determinar o melhor 
modelo. Porém, realizei testes superficiais e informais e o Bag of Words realmente parece ter o melhor desempenho dos 3.

# Determinação de threshold

O threshold foi escolhido novamente de maneira informal sobre o conjunto final.  
O valor fixado foi de `0.2`.

# Serialização do modelo

In [18]:
import pickle

In [19]:
X = data.TITLE.values
y = data.SMARTPHONE.values

In [20]:
model = bag_of_words
model.fit(X, y)
with open('model.clf', 'wb+') as f:
    serialized_model = pickle.dump(model, f)